# Moonbeam Quickstart (Google Colab, free GPU)

This notebook runs **end-to-end inference** with the pretrained **Moonbeam 309M** checkpoint and writes `out.mid` with **no dataset and no finetuning**.

## 1) Runtime setup
In Colab: `Runtime -> Change runtime type -> GPU`.

In [ ]:
import os
import subprocess

print(subprocess.check_output(["nvidia-smi"], text=True))

## 2) Clone repo and install dependencies (exact README commands)

In [ ]:
!git clone https://github.com/guozixunnicolas/Moonbeam-MIDI-Foundation-Model.git
%cd Moonbeam-MIDI-Foundation-Model
!pip install .
!pip install src/llama_recipes/transformers_minimal/.

## 3) Download pretrained checkpoint from Hugging Face

In [ ]:
from huggingface_hub import hf_hub_download

ckpt_path = hf_hub_download(
    repo_id="guozixunnicolas/moonbeam-midi-foundation-model",
    filename="moonbeam_309M.pt",
    local_dir="checkpoints/pretrained",
)
print("checkpoint:", ckpt_path)

## 4) Resolve config + tokenizer paths used by repo
- Model config: `src/llama_recipes/configs/model_config.json`
- Tokenizer: search for `tokenizer.model` in repo, fallback to benchmark tokenizer path.

In [ ]:
from pathlib import Path
import subprocess

repo_root = Path.cwd()
model_config_path = repo_root / "src/llama_recipes/configs/model_config.json"
assert model_config_path.exists(), f"Missing model config: {model_config_path}"

# Search for tokenizer.model in repo.
search = subprocess.run(
    ["bash", "-lc", "rg --files | rg 'tokenizer\.model$'"],
    cwd=repo_root,
    text=True,
    capture_output=True,
    check=False,
)
found = [line.strip() for line in search.stdout.splitlines() if line.strip()]
print("tokenizer.model candidates:", found)

if found:
    tokenizer_path = repo_root / found[0]
else:
    tokenizer_path = repo_root / "recipes/benchmarks/inference_throughput/tokenizer/tokenizer.model"

assert tokenizer_path.exists(), f"Missing tokenizer file: {tokenizer_path}"
print("using model_config_path:", model_config_path)
print("using tokenizer_path:", tokenizer_path)

## 5) Add dataset-free inference entrypoint (SOS-only prompt)
This avoids the existing CSV + `.npy` prompt requirement.

In [ ]:
# This file is committed in the fork at:
# recipes/inference/custom_music_generation/unconditional_from_scratch.py
!python recipes/inference/custom_music_generation/unconditional_from_scratch.py --help

## 6) Generate MIDI from scratch (no dataset)

In [ ]:
!python recipes/inference/custom_music_generation/unconditional_from_scratch.py   --ckpt_path "$ckpt_path"   --model_config_path src/llama_recipes/configs/model_config.json   --tokenizer_path "$tokenizer_path"   --output_midi_path out.mid   --max_seq_len 512   --max_gen_len 256   --temperature 0.9   --top_p 0.95

## 7) Verify output and (optional) render to audio preview

In [ ]:
from pathlib import Path

out_path = Path("out.mid")
assert out_path.exists() and out_path.stat().st_size > 0, "out.mid was not created"
print("✅ Generated:", out_path.resolve(), "size:", out_path.stat().st_size, "bytes")

In [ ]:
# Optional audio preview if dependencies are available.
# If synthesis backends are unavailable in Colab, this cell may be skipped.

!pip install pretty_midi midi2audio

import pretty_midi
from IPython.display import Audio, display

midi = pretty_midi.PrettyMIDI("out.mid")
# Attempt software synthesis (requires fluidsynth backend in runtime)
audio = midi.synthesize(fs=16000)
display(Audio(audio, rate=16000))

## Notes on checkpoint compatibility
`MusicLlama.build()` in this fork was updated to accept multiple checkpoint layouts:
- `{"model_state_dict": ...}`
- `{"state_dict": ...}`
- `{"model": ...}`
- or a raw state dict

It also strips `module.` prefixes when present.